# Step 9 Improvement in Dezero functions

## 9.1 Using Python Function

In [3]:
import numpy as np

class Variable:
    
    def __init__(self, data):
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
    
    def backward(self):
       funcs = [self.creator]
       while funcs:
           f = funcs.pop()
           x,y=f.input, f.output
           x.grad= f.backward(y.grad)

           if x.creator is not None:
               funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(y)
        output.set_creator(self) #set creator for the output variable   
        self.input=input #remember the input variable
        self.output=output #also save the output
        return output

class Square(Function):
    def forward(self, x):
        y=x**2
        return y
    
    def backward(self, gy):
        x=self.input.data
        gx=gy*2*x
        return gx
        
class Exp(Function):
    def forward(self, x):
        y=np.exp(x)
        return y
   
    def backward(self, gy):
        x=self.input.data
        gx=np.exp(x)*gy
        return gx

Dezero functions can be used as python function. 

In [4]:
def square(x):
    f=Square()
    return f(x)

def exp(x):
    f=Exp()
    return f(x)

Above can be written in short as following. 

In [5]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)
    

In [6]:
x=Variable(np.array(0.5))
a=square(x)
b=exp(a)
y=square(b)

y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


Above functions can be written in short as following. 

In [7]:
x=Variable(np.array(0.5))
y=square(exp(square(x)))
y.grad=np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 9.2 Simplification of backward method

As in the code cell [5], we have to write y.grad=np.array(1.0) everytime we want to do the back-propagation. To skip this part, we can add <br><br>
__if self.grad is None: self.grad=np.ones_like(self.data)__<br>


In [8]:
import numpy as np

class Variable:

    def __init__(self, data):
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
    
    def backward(self):

        if self.grad is None:
           self.grad=np.ones_like(self.data) #if grad is None, automatically generate the gradient

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y=f.input, f.output
            x.grad= f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

1. If grad is None, automatically generate the gradient.
2. np.ones_like creates the same datatype with self.data, which is the ndarray instance. 
3. np.ones_like fills out all the elements as 1. 
4. If self.data is scalar, self.grad is also scalar. 

Now we are able to get the gradient by only calling **backward()**. 

In [9]:
x=Variable(np.array(0.5))
y=square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


## 9.3 Dealing with ndarray exclusively

In [10]:
import numpy as np

class Variable:

    def __init__(self, data):
        
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        self.data=data
        self.grad=None
        self.creator=None
    
    def set_creator(self, func):
        self.creator=func
    
    def backward(self):

        if self.grad is None:
           self.grad=np.ones_like(self.data) #if grad is None, automatically generate the gradient

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x,y=f.input, f.output
            x.grad= f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [11]:
x=Variable(np.array(1.0))
x=Variable(None)

x=Variable(1.0)

TypeError: <class 'float'> is not supported

In [12]:
x=np.array([1.0])
y=x**2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


Above code works as it is expected. <br>However, __x__ becomes np.float64 when it is squared while it should be 0 dimensional ndarray.  

In [13]:
x=np.array(1.0)
y=x**2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


Using 0-dim ndarrays instances results in numpy.float64 or numpy.float32. 

Since we want our __Variable__ to always be ndarray type, we have to make the following function.

In [14]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

np.isscalar : function to check if input data is a scalar<br>
ex) __int__, __float__, __numpy.float64__

In [20]:
np.isscalar(np.float64(1.0))

True

In [21]:
np.isscalar(np.array(1.0))

False

In [22]:
np.isscalar(np.array([1,2,3]))

False

Hence, using the __as_array function__, we can convert the input data from scalar to ndarray instance. 

In [23]:
class Function:
    def __call__(self, input):
        x=input.data
        y=self.forward(x)
        output=Variable(as_array(y))
        output.set_creator(self) #set creator for the output variable   
        self.input=input #remember the input variable
        self.output=output #also save the output
        return output